# Mission to Mars

In [1]:
# import dependencies
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import time
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Choose the executable path to driver 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\tiffa\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [3]:
# this will be appended Mongo
scraped_data = {}

# NASA Mars News

In [4]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(1)

soup = BeautifulSoup(browser.html)
slide = soup.find("li", {"class": "slide"})
news_title = slide.find("div", {"class": "content_title"}).text.strip()
news_p = slide.find("div", {"class": "article_teaser_body"}).text.strip()

print(news_title)
print(news_p)

Celebrate the Perseverance Rover Landing With NASA's Student Challenge
The rover touches down on the Red Planet next month, and students are invited to join the excitement by designing, building, and landing their own Mars mission. NASA can help.


# JPL Mars Space Images - Featured Image

In [5]:
base = "https://www.jpl.nasa.gov"
url = f"{base}/spaceimages/?search=&category=Mars"
browser.visit(url)
time.sleep(1)

browser.find_by_id("full_image").click()
time.sleep(1)

browser.find_link_by_partial_text("more info").click()
time.sleep(1)

soup = BeautifulSoup(browser.html)
image = soup.find("img", {"class": "main_image"})

featured_image_url  = base + image["src"]

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16694_hires.jpg'

# Mars Facts

In [6]:
facts_url = 'http://space-facts.com/mars/'
mars_datas = pd.read_html(facts_url)

mars_facts = mars_datas[0]
mars_facts.columns = ['Description','Value']
mars_facts.set_index('Description', inplace=True)
mars_facts.to_html()
data = mars_facts.to_dict(orient='records')

mars_facts

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


# Hemisphere Data

In [7]:
base = "https://astrogeology.usgs.gov"
url = f"{base}/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)

soup = BeautifulSoup(browser.html)
links = soup.find("div", {"class": "results"}).findAll("a", {"class": "itemLink"})

#get rid of images
realLinks = []
for link in links:
    image = link.find("img")
    if (image):
        realLinks.append(base + link["href"]) 

#create the loop
hemisphere_data = []
for realLink in realLinks:
    browser.visit(realLink)
    time.sleep(1)
    
    soup = BeautifulSoup(browser.html)
    hemi_url = soup.find("ul").find("li").find("a")["href"]
    hemi_title = soup.find("h2", {'class', "title"}).text.split(" Enhanced")[0]
    
    hemisphere_data.append({"title": hemi_title, "url": hemi_url})

#exit browser
browser.quit()

# Appended Data

In [8]:
#append data
scraped_data["news_title"] = news_title
scraped_data["news_p"] = news_p
scraped_data["featured_image_url"] = featured_image_url
scraped_data["mars_facts"] = mars_facts
scraped_data["hemispheres"] = hemisphere_data

In [9]:
scraped_data

{'news_title': "Celebrate the Perseverance Rover Landing With NASA's Student Challenge",
 'news_p': 'The rover touches down on the Red Planet next month, and students are invited to join the excitement by designing, building, and landing their own Mars mission. NASA can help.',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16694_hires.jpg',
 'mars_facts':                                               Value
 Description                                        
 Equatorial Diameter:                       6,792 km
 Polar Diameter:                            6,752 km
 Mass:                 6.39 × 10^23 kg (0.11 Earths)
 Moons:                          2 (Phobos & Deimos)
 Orbit Distance:            227,943,824 km (1.38 AU)
 Orbit Period:                  687 days (1.9 years)
 Surface Temperature:                   -87 to -5 °C
 First Record:                     2nd millennium BC
 Recorded By:                   Egyptian astronomers,
 'hemispheres': [{'title